In [3]:
#collect calibration factor for the day
#must do this before collecting data for the day!!!

from ScopeChat import Oscope
import CalibrationFactor as cf
import Utilities as util

#record experiment variables 
optical_len = 2.00 #optical length
oven_temp = 145 #oven temp
laser_power = 0.400 #laser power on readout
laser_temp = 1.300 #laser temp on readout
laser_wavelen = 7.80542E-5 #laser wavelength

lockin_sensitivity = 0.02 #in VOLTS. THIS IS IMPORTANT

#set the rotation used for calibration (in degrees)
#the small knob on the rotation mount rotates one degree not 1/4th degree
#because Thor Labs chose violence when they decided how to make this mount
calibration_rotation = 1 #we want to use a full degree of rotation for this because we are 

#set up scope communication
#same as the actual data collection here
visa_address = 'USB0::0x0699::0x0368::C041014::INSTR'
source_channel = 'CH1'
my_scope = Oscope(visa_address, 16, source_channel)
my_scope.startScope()
my_scope.collectionSetUp()


cal_volts = my_scope.collectCalibrationMeasurement()

my_scope.scopeClose()

#now need to convert this to the actual value we we want. Right now it's a voltage diff
cal_info = cf.CalibrationFactor(calibration_rotation, lockin_sensitivity, cal_volts[0], cal_volts[1], cal_volts[2], cal_volts[3], laser_wavelen, optical_len, oven_temp, laser_power, laser_temp)
cal_info_conv = cal_info.calFPrepforJSON()

#now save calibration factors to their ownfile
ds = util.getDateString()
filename = 'Data/calibration_'+ds+'.json'
util.saveDatatoJSON(filename, cal_info_conv)



[1.22240889 1.22567415 1.25074029 1.24710679 1.25106037 1.2424649
 1.2462424  1.25277305 1.25389349 1.26681077 1.25954378 1.27580619
 1.26240885 1.26012003 1.27044404 1.25920761 1.26948369 1.26556206
 1.26756299 1.27100432]
[1.2560159325549995, 0.013349324475000035]
[1.27070022 1.26857138 1.27001202 1.26055217 1.26191282 1.27732682
 1.27806318 1.27278101 1.27547014 1.26892352 1.27577424 1.27623844
 1.26578629 1.26320934 1.26882744 1.25690269 1.26735485 1.26002395
 1.25554216 1.26805925]
[1.2681015968399998, 0.005630254750000008]
Calibration Factor [init, error, final, error]:  [1.25601593 0.01334932 1.2681016  0.00563025]


In [19]:
#collect rotation data
#does not collect calibration factor

from ScopeChat import Oscope
import densityCalculations as calc
import Utilities as util

#record experiment variables 
trial = 17
olen = 2.00 #optical length
t = 145 #oven temp
power = 0.400 #laser power on readout
laserTemp = 1.300 #laser temp on readout
laserwl = 7.80542E-5 #laser wavelength

#set timeconstant used on lockin
time_const = 1

#set the rotation used for calibration (in degrees)
#the small knob on the rotation mount rotates one degree not 1/4th degree
#because Thor Labs chose violence when they decided to that mount
#calibration_rotation = 0.16

#set up scope communication
visa_address = 'USB0::0x0699::0x0368::C041014::INSTR'
source_channel = 'CH1'
my_scope = Oscope(visa_address, 16, source_channel)
my_scope.startScope()
my_scope.collectionSetUp()

#removed for now - we want to use universal calibration factor for the day instead 
#update program later to standardize and improve this
#collect a calibration measurement
#calibration = my_scope.collectCalibrationMeasurement() #type quit to close

#collect data 
raw_data = my_scope.collectUntilDone(time_const)

#close the channel to the scope once done collecting data
my_scope.scopeClose()

#save the raw data to a file so I have if as backup
filename_raw = 'Data/rawdata.json'
raw = util.prepRawDataForFile(raw_data)
util.saveDatatoJSON(filename_raw, raw)


#set calibration factor to use for dataset
cal_factor = 0.0004206 #5mV scale on lock-in

#update 5/3/24 - using universal conversion factors now. 
#Will update program to take calibration independently of rest of data at a later time
conversion_f = cal_factor #calc.calculateRotationConversionFactor(calibration[0], calibration_rotation)

#now convert the raw data into analysis and save that to a separate file
ds = util.getDateString()
filename_converted = 'Data/rotationdata_'+ds+'.json'
converted_data = calc.convertRawDataArray(raw_data, conversion_f)
converted = util.prepConvertedDataforFile(converted_data, trial, laserwl, olen, t,power,conversion_f,laserTemp)
util.saveDatatoJSON(filename_converted, converted)

[-0.08366546 -0.0835094  -0.08328932 -0.08414966 -0.08307723 -0.08336534
 -0.08358143 -0.08513005 -0.0837495  -0.08338936 -0.08403761 -0.08491396
 -0.08483393 -0.08462185 -0.08561824 -0.08472589 -0.08578231 -0.08565025
 -0.08544617 -0.0847459 ]
[-0.08436414227000003, 0.0006762705750000021]
[-0.05581432 -0.05445778 -0.05515806 -0.05490596 -0.05417767 -0.05539415
 -0.05598639 -0.05328131 -0.05491797 -0.05460984 -0.05623049 -0.05581032
 -0.05604241 -0.05554222 -0.05686274 -0.0557623  -0.05656262 -0.05640656
 -0.05602241 -0.05691876]
[-0.055543214815000005, 0.0009093629250000006]
[-0.03544618 -0.03660664 -0.03486995 -0.03612245 -0.0354942  -0.03567427
 -0.03590236 -0.03619848 -0.03561024 -0.03548619 -0.03577031 -0.03527811
 -0.03486595 -0.03460184 -0.03490196 -0.03538615 -0.03488195 -0.03462985
 -0.03423769 -0.03445378]
[-0.035320926834999995, 0.0005922363999999996]
[-0.02233293 -0.02234894 -0.02290116 -0.02213285 -0.02279712 -0.02212085
 -0.022505   -0.02234494 -0.02219288 -0.0227451  -0.